## Lyzr Content Creation Assistant

This notebook provides a detailed guide and fully documented code examples to build a Content Creation Assistant using the Lyzr Agent API.

**1.  Installation of Lyzr Agent API**

First install the lyzr-agent-api with the help of pip command.


In [1]:
!pip install lyzr-agent-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


**2.  Setup and Initialization**

Import the necessary library and initialize your Lyzr API client.

In [2]:
from lyzr_agent_api import AgentAPI, AgentConfig
from lyzr_agent_api import EnvironmentConfig, FeatureConfig
from lyzr_agent_api import ChatRequest, OpenAPISchema

In [3]:
# X_API_KEY variable holds the Lyzr api key to authenticate the api requests, basically its a header token.
X_API_Key = "X_API_Key"

In [4]:
# OPENAI_API_KEY variable have our openai api key
OPENAI_API_KEY = "OPENAI_API_KEY"

In [5]:
# We need to initialize the client for AgentAPI.
client = AgentAPI(
    base_url="https://agent.api.lyzr.app",
    x_api_key=X_API_Key,
    timeout=3600
)

print("Client has been initialized successfully")

Client has been initialized successfully


**3. Environment Creation**

Set up the agent's environment.

The `create_env` function creates a new environment with specified configurations such as features as a module it uses `HUMANIZER`, tools such as `perplexity_search`, and a large language model (LLM) as `gpt-4o-mini`. It takes an environment name and optional API key, makes an API call to set up the environment, and returns the newly created environment's ID.

In [8]:
# This function will create the environment based on the given configuration
def create_env(env_name, openai_api_key=OPENAI_API_KEY):
    env_config = EnvironmentConfig(
        name=env_name,
        features=[
            FeatureConfig(
                type="HUMANIZER",
                config={},
                priority=0,
            )
        ],
        tools=["perplexity_search"],
        llm_config={"provider": "openai",
                    "model": "gpt-4o-mini",
                    "config": {
                        "temperature": 0.5,
                        "top_p": 0.9
                    },
                    "env": {
                        "OPENAI_API_KEY": openai_api_key
                    }},
    )

    response = client.create_environment_endpoint(json_body=env_config)
    if response:
      print('Environment has been created successfully')
      return response['env_id']

print("Environment is configured with HUMANIZER Module, tool as perplexity_search and LLM Model as gpt-4o-mini from OpenAI.")

Environment is configured with HUMANIZER Module, tool as perplexity_search and LLM Model as gpt-4o-mini from OpenAI.


**4. Agent Creation**

Define the configuration for your Content Creation Agent, outlining its purpose and capabilities in the system prompt.

The `create_agent` function creates an agent in a specified environment by taking the environment ID and agent name as inputs. It configures the agent with a system prompt for content creation tasks and an agent description, then makes an API call to create the agent, returning the newly created agent's ID.


In [9]:
# This function will help us create agent based on specific environment
def create_agent(env_id, agent_name):
    agent_config = AgentConfig(
        env_id=env_id,
        system_prompt=""" You are an expert content creation agent. You can generate different content formats like articles, social media posts, and code. """,
        name=agent_name,
        agent_description='Content Creator Assistant Agent'
    )

    agent_response = client.create_agent_endpoint(json_body=agent_config)
    if agent_response:
      print('Agent is configured successfully')
      return agent_response['agent_id']

print("Agent will need an environment id to configured successfully.")

Agent will need an environment id to configured successfully.


**5. User Interaction and Content Generation**

Create a function to handle user requests and generate content. This function will also include the logic for using the registered tools.

---



In [10]:
def generate_content(user_id, agent_id, user_request, content_format="article"):
    """Generates content based on user request and format.

    Args:
        user_request (str): The user's request for content (e.g., "Write a blog post about AI.").
        content_format (str, optional): The desired format of the content. Defaults to "article".

    Returns:
        str: The generated content.
    """

    json_body = ChatRequest(
        user_id=user_id, # Your user_id, you will get it from the studio.
        agent_id=agent_id,
        message=user_request,
    )

    chat = client.chat_with_agent(json_body=json_body)
    return chat['response']

In [ ]:
# Example Usage
user_id = 'user_id'
env_id = create_env(env_name='CookBook Content Creation Envrionment')
agent_id = create_agent(env_id=env_id, agent_name='CookBook Content Creation Agent')
user_request = "The benefits of using a content creation AI assistant."
generated_content = generate_content(user_id=user_id,
                                     agent_id=agent_id,
                                     user_request= user_request,
                                     content_format="blog post")


**Key Points and Further Enhancements:**

- **Prompt Engineering:** Experiment with different prompts to guide the agent to generate specific content styles or tones. For example, you can ask for a "formal" article or a "humorous" social media post.
- **Content Variety:** Expand the agent's capabilities by adding more content formats (e.g., scripts, poems, code in different programming languages) and integrating relevant tools.
- **Iterative Content Refinement:**  Allow users to provide feedback on the generated content and have the agent refine it iteratively.

In [ ]:
print(f"Generated Content:\n{generated_content}")